# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**: José Juan Díaz Campos

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from jjodiaz.spark_utils import SparkUtils
from pyspark.sql.functions import get_json_object, col

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("local[*]") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/23 14:25:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Leer los csv

In [2]:
schema_agencies = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])
df_agencies = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/agencies")

schema_brands = SparkUtils.generate_schema([("brand_id", "int"), ("brand_info", "string")])
df_brands = spark.read.schema(schema_brands).option("header", True).csv("/opt/spark/work-dir/data/car_service/brands")

schema_cars = SparkUtils.generate_schema([("car_id", "int"), ("car_info", "string")])
df_cars = spark.read.schema(schema_cars).option("header", True).csv("/opt/spark/work-dir/data/car_service/cars")

schema_customers = SparkUtils.generate_schema([("customer_id", "int"), ("customer_info", "string")])
df_customers = spark.read.schema(schema_customers).option("header", True).csv("/opt/spark/work-dir/data/car_service/customers")

schema_rentals = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])
df_rentals = spark.read.schema(schema_rentals).option("header", True).csv("/opt/spark/work-dir/data/car_service/rentals")

## Hacer el join

In [3]:
final_result = df_rentals \
    .withColumn("car_id", get_json_object(col("rental_info"), "$.car_id").cast("int")) \
    .withColumn("customer_id", get_json_object(col("rental_info"), "$.customer_id").cast("int")) \
    .withColumn("agency_id", get_json_object(col("rental_info"), "$.agency_id").cast("int")) \
    .join(
        df_cars.withColumn("car_name", get_json_object(col("car_info"), "$.car_name")),
        "car_id", 
        "inner"
    ) \
    .join(
        df_customers.withColumn("customer_name", get_json_object(col("customer_info"), "$.customer_name")),
        "customer_id", 
        "inner"
    ) \
    .join(
        df_agencies.withColumn("agency_name", get_json_object(col("agency_info"), "$.agency_name")),
        "agency_id", 
        "inner"
    ) \
    .select(
        col("rental_id"),
        col("car_name"),
        col("agency_name"),
        col("customer_name")
    ) \
    .orderBy("rental_id")


In [4]:
final_result.show(20, truncate=False)

[Stage 3:>                                                          (0 + 7) / 7]

+---------+------------------------------------+-------------+----------------+
|rental_id|car_name                            |agency_name  |customer_name   |
+---------+------------------------------------+-------------+----------------+
|0        |Grimes-Green Model 8                |SF Cars      |Laura Perry     |
|1        |Wallace-Carlson Model 9             |SF Cars      |Melanie Patrick |
|2        |Wagner LLC Model 1                  |NYC Rentals  |Theresa Estrada |
|3        |Bolton, Burns and Turner Model 10   |LA Car Rental|Amanda Garcia   |
|4        |Alvarez-Davis Model 3               |SF Cars      |Corey Cook      |
|5        |Grimes-Green Model 8                |SF Cars      |Kyle Ramos      |
|6        |Grimes-Green Model 8                |NYC Rentals  |Julie Chen      |
|7        |Clayton-Cook Model 10               |LA Car Rental|Sarah Peterson  |
|8        |Garcia, Hamilton and Carr Model 5   |SF Cars      |Jay Walsh       |
|9        |Chang-Fisher Model 7         

In [ ]:
sc.stop()